# Приветствую !

Это небольшая инструкция по использованию функционала для автоматического перебора пайплайнов в DeepPavlov. 

### Зачем это нужно ?

Конкретный пример. Решается задача классификации, допустим интентов. У вас есть 10 моделей, которые могут какой-то результат, несколько токенайзеров, опечаточник, лемматизатор, ELMo, fasttext, и много чего ещё. И по хорошему, вам бы попробовать все модели, желательно ещё и с разными комбинациями векторизации и препроцессинга. После чего сформировать некий отчёт по проведённым экспериментам. Чтобы наглядно увидеть какая модель работает лучше, что круче ELMo или fasttext, и т.д.

Представленный ниже функционал позволяет автоматизировать, в некоторой мере, вышеперечисленные задачи, и облегчить вам жизнь (но это не точно).

## Описание основного класса

"pipeline_manager" - это специальный класс, который берёт на вход путь к конфиг файлу, и ряд дополнительных параметров, которые задают его режим работы. В конфиг файле задан некий "шаблон перебора" пайплайнов, на основании которого специальный генератор внутри класса, комбинируя различные компоненты указанные в шаблоне перебора, выдаёт набор конфигов в стандартном для DeepPavlov-а виде. Дальше каждый такой конфиг отдельно запускается, обучается и тестируется при помощи функционала библиотеки. По мере обучения и тестирования различных пайплайнов когнфиги пайплайнов и промежуточные результаты логируются в отдельном файлике. По окончании работы алгоритма, в папке Downloads появится новая папка experiments, в которой будут содержатся логи, веса моделей, а также excel файл с отображением всех исполненных пайплайнов с описанием компонент, и достигнутых значений на метриках.

In [ ]:
# Сам класс располагается по следующему пути
from deeppavlov.pipeline_manager.pipeline_manager import PipelineManager

# И экземпляр класса создаётся следующим образом
manager = PipelineManager(config_path, exp_name, mode, info, root, hyper_search, sample_num, target_metric)

# Запускается эксперимент так
manager.run()

**config_path** - str: Путь к конфиг файлу, в котором задан шаблон перебора пайплайнов, параметры обучения, и т.д. это обязательный параметр без значения по дефолту

**exp_name** - str: Имя эксперимента, требуется для формирования логов и отчета, это обязательный параметр без значения по дефолту

**mode** - str: [train, evaluate] режим работы pipeline manager, может как обучать данные, так и тестить уже обученные модели, train используется по умолчанию

**info** - Dict or None: Просто некая дополнительная информация, которую вы хотите сохранить в логах в виде словаря. Информация может быть любой, этот параметр не влияет на работу pipeline manager. По умолчанию принимает значение None.

**root** - str: Путь к папке в которой будет формироваться отчет, и создаться папка experiments, значение './experiments/' используется по умолчанию

**hyper_search** - str: [grid, random] триггер указывающий какой тип поиска, grid используется по умолчанию

**sample_num** - int: Если hyper_search=random, то sample_num указывает количество сгенерированных примеров. Значение по дефолту 10.

**target_metric** - str: Имя метрики на основании которой будет осуществляться сортировка результатов при формировании отчёта. Значение по дефолту None, в таком случае в качестве целевой метрики берется первая из тех имён что указаны в конфиг файле. Если указанной метрики нет в DeepPavlov, вызовется исключение.


## Создание конфиг файла

Конфиг, путь к которому подаётся на вход классу pipeline manager, отличается от "стандартного" конфига, который требуется написать для запуска некой модели в DeepPavlov, лишь тем, как в нём описывается содержание поля "pipe" в чейнере. Вот краткая версия того как в чейнере задаётся поле "pipe":

In [ ]:
{"chainer": {"in": ["x"], "in_y": ["y"],
             "pipe": [
                       {"id": "classes_vocab",
                        ...},
                       {"id": "my_embedder",
                        ...},
                       {"in": ["x"],
                        "name": "str_lower",
                        "out": ["x"]},
                       {"id": "my_tokenizer",
                         ...},
                       {"in": ["x"], "in_y": ["y"], "out": ["y_labels", "y_probas_dict"],
                        "main": True,
                        "name": "intent_model",
                        ...},
                     ],
             "out": ["y_labels", "y_probas_dict"]}}

Как видно в примере, мы задаём поле "pipe" в виде списка словарей, которые явно описывают последовательность компонент и их параметров. В то время как для автоматического запуска множества разных пайплайнов, нам требуется задать шаблон перебора разных компонент. Для этого каждый элемент списка в поле "pipe" трансформируется в отдельный список словарей, в котором идёт перечисление компонент с параметрами, которые требуются для перебора. Даже если на некой позиции в пайплайне расположен всего один компонент, то он должен быть обёрнут в список. Вот пример:

In [ ]:
{"chainer": {"in": ["x"], "in_y": ["y"],
    "pipe": [
      [{"id": "classes_vocab",
        ...}],
      [{"id": "my_embedder",
        ...}],
      [{"name": "str_lower", ...}, None],
      [{"name": "nltk_tokenizer",
        ...},
       {"name": "lazy_tokenizer",
        ...}],
      [{"name": "intent_model",
        "model_name": "cnn_model",
        ...},
       {"name": "intent_model",
        "model_name": "bilstm_model",
        ...}]
       ],
    "out": ["y_labels", "y_probas_dict"]}
}

Как можно видеть теперь у нас в конфиге два токенайзера, и две модели CNN и bi-LSTM.Также обратите внимание что третий элемент кроме словаря также содержит None:

In [ ]:
"pipe": [...
         [{"name": "str_lower", ...}, None],
         ...]

Это значит, что этот элемент может и вовсе отсутствовать в пайплайне. В итоге будут созданы пайплайны не содержащие этого компонента. В данном случае несложно посчитать, что в эксперименте будет запущено 8 разных пайплайнов. 

Вот в общем-то и всё. Для того чтобы задать шаблон перебора вам требуется взять ваш конфиг, который вы уже использовали до этого и дописать в "pipe" chainer-a дополнительные компоненты и модели, которые вы хотите попробовать. Главное не забыть что каждый элемент в новом "pipe" является отдельным списком. Потом уже можно запустить эксперимент через командную строку, введя:

### python -m deeppavlov sort_out  <путь к новому конфиг файлу>  -e  <имя эксперимента>

По окончании в папке downloads появится папка experiments, в которой будут сохраняться все ваши данные, отчёты, чекпоинты по отдельным экспериментам, рассортированные по датам и именам экспериментов.

Может возникнуть такая ситуация, что в рамках одного конфига вы не сможете описать все желаемые варианты пайплайнов, из-за несовместимости отдельных компонент. В таком случае вы можете создать два конфига, и поочерёдно запустить их, указав одно и тоже имя эксперимента, в таком случае, все логи и отчёты, автоматически соединяться.

## Подбор гиперпараметров

Также pipeline manager может осуществлять подбор гиперпараметров как основной модели, так и отдельных компонент, если требуется. На данный момент поддерживается "random" или "grid" поиск. При поиске оптимальных гиперпараметров, количество различных пайплайнов, а следовательно и время испольнения алгоритма, может существенно увеличится, это надо иметь в виду.

### Grid search

Для запуска подбора гиперпараметров требуется в конфиге указать для каких именно компонент требуется осуществлять подбор. Это делается за счёт добавления в словарь, описывающий компоненту и ее параметры, пары ключ-значение -  "search": True; После чего нужным параметрам сопоставить список со значениями для перебора.

**ВАЖНОЕ ЗАМЕЧАНИЕ:**
Если в исходном словаре имеются имеются ключи, значениями которых являются списки, то им следует добавить ещё один уровень вложенности. (См. пример ниже)

Вот пример словаря по которому можно осуществлять grid search.

In [ ]:
{"search": True,
 "in": [["x"]], "in_y": [["y"]], "out": [["y_labels", "y_probas_dict"]],
 "main": True,
 "scratch_init": True,
 "name": "intent_model",
 "save_path": "intents/intent_cnn_snips_v4",
 "classes": "#classes_vocab.keys()",
 "kernel_sizes_cnn": [[1, 2, 3], [1, 3, 6]],
 "filters_cnn": [64, 128, 256],
 "confident_threshold": [0.1, 0.3, 0.5, 0.7],
 "optimizer": "Adam",
 "lear_rate": [0.01, 0.001],
 "lear_rate_decay": 0.1,
 "loss": "binary_crossentropy",
 "text_size": [15, 20, 25, 40],
 "coef_reg_cnn": 1e-4,
 "coef_reg_den": 1e-4,
 "dropout_rate": 0.5,
 "dense_size": [50, 100, 150, 200],
 "model_name": "cnn_model",
 "embedder": "#my_embedder",
 "tokenizer": "#my_tokenizer"}

### Random search

Здесь по большому счёту ситуация такая же. Однако если grid search осуществит перебор всех возможных комбинаций параметров, то random search семплирует случайные наборы параметров из указанных в конфиге диапазонов определённое количество раз. За это отвечает параметр sample_num в pipeline manager. Добавление вложенности при наличии в словаре списков в качестве значений, здесь также необходимо.

Функционал при Random search несколько больше. Если вы вы параметру присвоите список значений, то при семплирования будет случайно будет выбран один из его элементов. Однако параметру можно также сопоставить и словарь инструкциями:

**"some_parameter": {"bool": True}** - Случайно присвоит True или False

**"some_parameter": {"range": [start, end]}** - Присваивает случайное значение из указанного диапазона

**"some_parameter": {"range": [start, end], "scale": "log"}** - Присваивает случайное значение из указанного диапазона, но в логарифмическом масштабе.

**"some_parameter": {"range": [start, end], "discrete": True}** - Присваивает случайное, и округлённое, значение из указанного диапазона

Вот пример:

In [ ]:
{"search": True,
 "in": [["x"]], "in_y": [["y"]], "out": [["y_labels", "y_probas_dict"]],
 "main": True,
 "scratch_init": True,
 "name": "intent_model",
 "save_path": "intents/intent_cnn_snips_v4",
 "classes": "#classes_vocab.keys()",
 "kernel_sizes_cnn": [[1, 2, 3], [1, 3, 6]],
 "filters_cnn": {'range': [64, 512], 'discrete': True},
 "confident_threshold": 0.5,
 "optimizer": "Adam",
 "lear_rate": {'range': [0.1, 0.0001], 'scale': 'log'},
 "lear_rate_decay": 0.1,
 "loss": "binary_crossentropy",
 "text_size": 15,
 "coef_reg_cnn": 1e-4,
 "coef_reg_den": 1e-4,
 "dropout_rate": 0.5,
 "dense_size": 100,
 "model_name": "cnn_model",
 "embedder": "#my_embedder",
 "tokenizer": "#my_tokenizer"
}